<a href="https://colab.research.google.com/github/mohamedalaaaz/EmailModule/blob/main/2D%20Maxwell%20PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.autograd as autograd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Optionally: RFF mapping layer for better learning
class RFF(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.B = nn.Parameter(torch.randn(in_features, out_features) * 10, requires_grad=False)
        self.scale = 1.0

    def forward(self, x):
        return torch.cat([torch.sin(self.scale * x @ self.B), torch.cos(self.scale * x @ self.B)], dim=1)

class MaxwellPINN2D(nn.Module):
    def __init__(self, use_rff=False, rff_dim=128):
        super().__init__()
        self.use_rff = use_rff
        if use_rff:
            self.rff = RFF(3, rff_dim)
            input_dim = 2 * rff_dim
        else:
            input_dim = 3
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 3)  # Outputs: [Ez, Hx, Hy]
        )

    def forward(self, x, y, t):
        inp = torch.cat([x, y, t], dim=1)
        if self.use_rff:
            inp = self.rff(inp)
        return self.net(inp)

def maxwell_loss(model, x, y, t):
    x.requires_grad_(True); y.requires_grad_(True); t.requires_grad_(True)
    output = model(x, y, t)
    Ez, Hx, Hy = output[:, [0]], output[:, [1]], output[:, [2]]

    # Compute derivatives
    Ez_x = autograd.grad(Ez, x, torch.ones_like(Ez), create_graph=True)[0]
    Ez_y = autograd.grad(Ez, y, torch.ones_like(Ez), create_graph=True)[0]
    Ez_t = autograd.grad(Ez, t, torch.ones_like(Ez), create_graph=True)[0]

    Hx_t = autograd.grad(Hx, t, torch.ones_like(Hx), create_graph=True)[0]
    Hy_t = autograd.grad(Hy, t, torch.ones_like(Hy), create_graph=True)[0]
    Hx_y = autograd.grad(Hx, y, torch.ones_like(Hx), create_graph=True)[0]
    Hy_x = autograd.grad(Hy, x, torch.ones_like(Hy), create_graph=True)[0]

    # Residuals for Maxwell’s equations in vacuum (2D TE mode)
    r1 = Ez_x + Hx_t     # ∂Ez/∂x + ∂Hx/∂t = 0
    r2 = Ez_y - Hy_t     # ∂Ez/∂y - ∂Hy/∂t = 0
    r3 = Hx_y - Hy_x + Ez_t  # ∂Hx/∂y - ∂Hy/∂x + ∂Ez/∂t = 0

    loss = torch.mean(r1**2 + r2**2 + r3**2)
    return loss

# Training procedure
def train(model, epochs=5000, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        x = torch.rand(1024,1,device=device)
        y = torch.rand(1024,1,device=device)
        t = torch.rand(1024,1,device=device)

        loss = maxwell_loss(model, x, y, t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")


In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Create a grid in (x, y) at fixed time t0
t0 = 0.5
nx, ny = 100, 100
x_vals = torch.linspace(0, 1, nx, device=device)
y_vals = torch.linspace(0, 1, ny, device=device)
X, Y = torch.meshgrid(x_vals, y_vals, indexing='ij')

x_flat = X.reshape(-1, 1)
y_flat = Y.reshape(-1, 1)
t_flat = torch.full_like(x_flat, t0)

with torch.no_grad():
    output = model(x_flat, y_flat, t_flat).cpu()
    Ez = output[:, 0].reshape(nx, ny).numpy()
    Hx = output[:, 1].reshape(nx, ny).numpy()
    Hy = output[:, 2].reshape(nx, ny).numpy()

# Plot Ez field
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title(f"Electric Field Ez at t={t0}")
plt.imshow(Ez, extent=[0,1,0,1], origin='lower', cmap='RdBu')
plt.colorbar()

# Plot Hx field
plt.subplot(1, 3, 2)
plt.title(f"Magnetic Field Hx at t={t0}")
plt.imshow(Hx, extent=[0,1,0,1], origin='lower', cmap='RdBu')
plt.colorbar()

# Plot Hy field
plt.subplot(1, 3, 3)
plt.title(f"Magnetic Field Hy at t={t0}")
plt.imshow(Hy, extent=[0,1,0,1], origin='lower', cmap='RdBu')
plt.colorbar()

plt.tight_layout()
plt.show()


NameError: name 'model' is not defined